In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score


In [4]:
df = pd.read_csv("Churn_Modelling.csv")  


In [5]:

df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])

df['Geography'] = LabelEncoder().fit_transform(df['Geography'])
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])

X = df.drop("Exited", axis=1)
y = df["Exited"]

X = StandardScaler().fit_transform(X)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
xgb_default = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_default.fit(X_train, y_train)
y_pred_default = xgb_default.predict(X_test)
acc_default = accuracy_score(y_test, y_pred_default)


c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:51:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [8]:
param_grid = {
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [50, 100]
}

grid = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)

grid.fit(X_train, y_train)
best_params = grid.best_params_
best_model = grid.best_estimator_

y_pred_best = best_model.predict(X_test)
acc_best = accuracy_score(y_test, y_pred_best)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:51:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [9]:
print(" Accuracy (Default):", round(acc_default, 3))
print(" Accuracy (Tuned):", round(acc_best, 3))
print(" Best Parameters:", best_params)


 Accuracy (Default): 0.856
 Accuracy (Tuned): 0.866
 Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
